In [1]:
import psycopg2
conn = psycopg2.connect(host="localhost", dbname="postgres", user="postgres", password="1234")

In [ ]:
from pathlib import Path
import json

# Pfad anpassen
json_path = Path("../station_data.json")

# Variante A: read_text + json.loads
raw = json.loads(json_path.read_text(encoding="utf-8"))
# (Falls du mal Probleme mit BOM hast: encoding="utf-8-sig")

# Falls die DB-JSON ein dict mit "result" ist:
stations = raw["result"] if isinstance(raw, dict) and "result" in raw else raw
print("Stations in file:", len(stations))

rows = []
for st in stations:
    name = st.get("name")
    for eva in (st.get("evaNumbers") or []):
        if eva.get("isMain") is True:
            eva_no = eva.get("number")

            coords = (eva.get("geographicCoordinates") or {}).get("coordinates")
            lon = lat = None
            if isinstance(coords, (list, tuple)) and len(coords) >= 2:
                lon, lat = float(coords[0]), float(coords[1])

            if eva_no is not None:
                rows.append(
                    {
                        "eva": int(eva_no),
                        "name": name,
                        "lat": lat,
                        "lon": lon,
                    }
                )

print("Main EVA rows:", len(rows))
print("First 5:", rows[:5])

create_sql = """
CREATE TABLE IF NOT EXISTS stationen (
    eva        BIGINT PRIMARY KEY,
    name       TEXT NOT NULL,
    latitude   DOUBLE PRECISION,
    longitude  DOUBLE PRECISION
);
"""

with conn.cursor() as cur:
    cur.execute(create_sql)
conn.commit()

print("Table 'stationen' ensured.")

upsert_sql = """
INSERT INTO stationen (eva, name, latitude, longitude)
VALUES (%s, %s, %s, %s)
ON CONFLICT (eva) DO UPDATE
SET name = EXCLUDED.name,
    latitude = EXCLUDED.latitude,
    longitude = EXCLUDED.longitude;
"""

payload = [(r["eva"], r["name"], r["lat"], r["lon"]) for r in rows]

with conn.cursor() as cur:
    cur.executemany(upsert_sql, payload)
conn.commit()

print("Upserted rows:", len(payload))

with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM stationen;")
    print("Rows in stationen:", cur.fetchone()[0])

    cur.execute("SELECT eva, name, latitude, longitude FROM stationen ORDER BY eva LIMIT 10;")
    print(cur.fetchall())

conn.close()

Stations in file: 133
Main EVA rows: 133
First 5: [{'eva': 8011003, 'name': 'Ahrensfelde', 'lat': 52.571375, 'lon': 13.565154}, {'eva': 8080040, 'name': 'Albrechtshof', 'lat': 52.549396, 'lon': 13.128917}, {'eva': 8011155, 'name': 'Alexanderplatz', 'lat': 52.521481, 'lon': 13.410961}, {'eva': 8089054, 'name': 'Altglienicke', 'lat': 52.407299, 'lon': 13.558753}, {'eva': 8089003, 'name': 'Attilastraße', 'lat': 52.447681, 'lon': 13.360951}]
Table 'stationen' ensured.
Upserted rows: 133
Rows in stationen: 133
[(8010035, 'Berlin-Karlshorst', 52.4797942, 13.5279465), (8010036, 'Berlin-Lichtenberg', 52.5101355, 13.4966934), (8010041, 'Berlin-Schöneweide Pbf', 52.455127, 13.509395), (8010255, 'Berlin Ostbahnhof', 52.5104881, 13.4346807), (8010403, 'Berlin-Charlottenburg', 52.5042126, 13.3023298), (8010404, 'Berlin-Spandau', 52.5346481, 13.1968975), (8010405, 'Berlin-Wannsee', 52.4209723, 13.1795275), (8010406, 'Berlin Zoologischer Garten', 52.5073407, 13.3324447), (8011003, 'Ahrensfelde', 52.5